In [ ]:
import tensorflow as tf
from tensorflow.keras import *
import  pandas as pd
from sklearn import*
import numpy as np
import numpy
from keras.callbacks import ModelCheckpoint
from keras.layers.core import Dense, Activation, Dropout
from keras.models import Model
from keras.layers import add,Input,Conv1D,Activation,Lambda,Dense,Flatten
import inspect
from typing import List
from tensorflow.keras import backend as K, Model, Input, optimizers
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation, SpatialDropout1D, Lambda
from tensorflow.keras.layers import Layer, Conv1D, Dense, BatchNormalization, LayerNormalization
from keras.layers import Layer
from keras.callbacks import ModelCheckpoint
import keras.backend as K
from keras.layers import Multiply,Dropout,Bidirectional,LSTM
from keras.layers.core import *
from keras.models import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import h5py as hp
import math
from keras import optimizers
from sklearn.metrics import confusion_matrix
import scipy.io as sci  
from pandas import read_csv
from datetime import datetime
import numpy
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.layers import LSTM,GRU
import  pandas as pd
import  os
import time
from keras.models import Sequential, load_model
from sklearn import*
from keras.layers.core import Dense, Activation, Dropout
import random
from keras import layers
from keras.layers.core import Lambda
from tensorflow import keras
from tensorflow.keras import layers, models

filename='./Target intention recognition dataset.csv'
dataset = pd.read_csv('Target intention recognition dataset.csv', engine='python')

dataset1 = pd.read_csv(filename,usecols=[4,5,6,7,8,9,10,11,12,13,15,17],engine='python')
ndarray1=dataset1.values
Features=ndarray1[:,0:12]

ndarray=dataset.values
minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
scaledFeatures=minmax_scale.fit_transform(Features)
slid=12
label=[]
a=ndarray[:,3]
num=len(ndarray)-slid+1
for i in range(num):
    c=sum(a[i:i+slid])/slid
    if (c<=50):
        d=0
    elif(c>50)and(c<=80):
        d=1
    elif(c>80)and(c<=110):
        d=2
    elif(c>110)and(c<=150):
        d=3
    elif(c>150)and(c<=200):
        d=4
    else:
        d=5
    label.append(d)
scale=0.8
s=int(scale*num)
trainX=[]
testX=[]
for i in range(s):
    e=scaledFeatures[i:slid+i,:]
    trainX.append(e)
for i in range(s,num):
    e=scaledFeatures[i:slid+i,:]
    testX.append(e)
from keras.utils import to_categorical
onehot = to_categorical(label, num_classes=7)
trainY=onehot[:s,:]
testY=onehot[s:,:]
trainX=numpy.array(trainX)
trainY=numpy.array(trainY)
testX=numpy.array(testX)
testY=numpy.array(testY)
trainX =trainX.reshape(s, slid, 12)
trainY=trainY.reshape(s,7)
testX=testX.reshape(num-s,slid,12)
testY=testY.reshape(num-s,7)
print(trainX.shape,trainY.shape,testX.shape,testY.shape)

def attention_3d_block(inputs):
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, TIME_STEPS))(a)
    a = Dense(TIME_STEPS, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul
  
def model_attention_applied_after_lstm():
    K.clear_session()
    inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    lstm1 = Bidirectional(GRU(units=256, return_sequences=True))(inputs)
    drop1 = Dropout(0.5)(lstm1)
    lstm2 = Bidirectional(GRU(units=128,return_sequences=True))(drop1)
    drop2 = Dropout(0.5)(lstm2)
    lstm3 = Bidirectional(GRU(units=128, return_sequences=True))(drop2)
    drop3 = Dropout(0.5)(lstm3)
    attention_mul = attention_3d_block(drop3)
    attention_mul = Flatten()(attention_mul)
    output = Dense(7, activation='softmax')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model
    
def model_attention_applied_before_lstm():
    K.clear_session()
    inputs = Input(shape=(TIME_STEPS, INPUT_DIM,))
    attention_mul = attention_3d_block(inputs)
    
    attention_mul = LSTM(lstm_units, return_sequences=False)(attention_mul)
    output = Dense(7, activation='softmax')(attention_mul)
    model = Model(input=[inputs], output=output)
    return model
SINGLE_ATTENTION_VECTOR = True
APPLY_ATTENTION_BEFORE_LSTM = False
INPUT_DIM = 12
TIME_STEPS = 12
epochs = 200
batch_size = 512

if __name__ == '__main__':
    train_x = trainX
    train_y = trainY
    test_x = testX
    test_y = testY
    starttime=int(time.time())
    m.compile('Nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    m.summary()
    filepath='stabc.best.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True,mode='max')
    callbacks_list = [checkpoint]
    history=m.fit(train_x, train_y,validation_data=(testX, testY),epochs=epochs, batch_size=batch_size,callbacks=callbacks_list, verbose=2)
    endtime=int(time.time())
    mod = load_model('stabc.best.hdf5')
    predictions = mod.predict(test_x)
    conf = confusion_matrix(test_y.argmax(axis=1), predictions.argmax(axis=1))
    print(conf)

def attention_3d_block(inputs):
    input_dim = int(inputs.shape[2])
    a = inputs
    a = Dense(input_dim, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x,axis=1),name='dim_reduction')(a)
        a = RepeatVector(TIME_STEPS)(a)
    a_probs = Permute((1, 2), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a_probs])
    return output_attention_mul

SINGLE_ATTENTION_VECTOR = True
TIME_STEPS = 12

batch_size = 256
epochs = 200
filter_nums =7
kernel_size = 4

train_X, train_label, test_X, test_label =trainX,trainY,testX,testY
A=np.zeros((6,200))
B=np.zeros((6,200))
C=np.zeros((6,200))
D=np.zeros((6,200))


for i  in range(5):
    model = models.Sequential()
    input_tensor = Input(shape=(12,12))
    attention_mul = attention_3d_block(input_tensor)
    x = stabc(nb_filters=filter_nums, kernel_size=kernel_size, dilations=[1, 2],return_sequences=True)(attention_mul)
    GRU1 = layers.Bidirectional(layers.GRU(units=128, return_sequences=True))(x)
    drop1 = Dropout(0.5)(GRU1 )
    GRU2 = layers.Bidirectional(layers.GRU(units=64, return_sequences=False))(drop1)
out_put = layers.Dense(units=7, activation='softmax')(GRU2)
    model = models.Model(inputs=input_tensor, outputs=[out_put])
    model.summary()
    filepath='weights.best.hdf5'
    checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True,mode='max')
    callbacks_list = [checkpoint]
    nadam=optimizers.Nadam(lr=0.004, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004)
    model.compile(optimizer=nadam, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    train_history=model.fit(train_X, train_label, validation_data=(test_X, test_label), epochs=epochs, verbose=2, callbacks=callbacks_list)
    c=np.array(train_history.history['val_categorical_accuracy'])
    d=np.array(train_history.history['val_loss'])
    a=np.array(train_history.history['categorical_accuracy'])
    b=np.array(train_history.history['loss'])  
    A[i,:]=a
    B[i,:]=b
    C[i,:]=c
    D[i,:]=d
test_X[1:10].shape
a=[]
for i in range(100):
    starttime =time.time()
    predictions = model.predict(test_X[100:101])
    endtime = time.time()
    aa=(endtime-starttime)/1
    a.append(aa)
x=sum(a)/100
print(x)
fn5=(A[0,0,:]+A[0,1,:]+A[0,2,:])/3
fn7=(A[1,0,:]+A[1,1,:]+A[1,2,:])/3
fn10=(A[2,0,:]+A[2,1,:]+A[2,2,:])/3
fn15=(A[3,0,:]+A[3,1,:]+A[3,2,:])/3
fn20=(A[4,0,:]+A[4,1,:]+A[4,2,:])/3
print(max(fn5),max(fn7),max(fn10),max(fn15),max(fn20))
lfn5=(B[0,0,:]+B[0,1,:]+B[0,2,:])/3
lfn7=(B[1,0,:]+B[1,1,:]+B[1,2,:])/3
lfn10=(B[2,0,:]+B[2,1,:]+B[2,2,:])/3
lfn15=(B[3,0,:]+B[3,1,:]+B[3,2,:])/3
lfn20=(B[4,0,:]+B[4,1,:]+B[4,2,:])/3
print(min(lfn5),min(lfn7),min(lfn10),min(lfn15),min(lfn20))
plt.plot(fn7) 
kz2=(C[0,0,:]+C[0,1,:]+C[0,2,:])/3
kz3=(C[1,0,:]+C[1,1,:]+C[1,2,:])/3
kz4=(C[2,0,:]+C[2,1,:]+C[2,2,:])/3
kz5=(C[3,0,:]+C[3,1,:]+C[3,2,:])/3
kz6=(C[4,0,:]+C[4,1,:]+C[4,2,:])/3
print(max(kz2),max(kz3),max(kz4),max(kz5),max(kz6))
lkz2=(D[0,0,:]+D[0,1,:]+D[0,2,:])/3
lkz3=(D[1,0,:]+D[1,1,:]+D[1,2,:])/3
lkz4=(D[2,0,:]+D[2,1,:]+D[2,2,:])/3
lkz5=(D[3,0,:]+D[3,1,:]+D[3,2,:])/3
lkz6=(D[4,0,:]+D[4,1,:]+D[4,2,:])/3
print(min(lkz2),min(lkz3),min(lkz4),min(lkz5),min(lkz6))

List2=[128,256,512]
train_X, train_label, test_X, test_label =trainX,trainY,testX,testY
t=0
E=np.zeros((3,3,200))
F=np.zeros((3,3,200))
for i  in List2:
    hidden = i
    for j in range(3):
        model = models.Sequential()
        input_tensor = Input(shape=(12,12))
        attention_mul = attention_3d_block(input_tensor)
        x = stabc(nb_filters=filter_nums, kernel_size=kernel_size, dilations=[1, 2], return_sequences=True)(attention_mul)
        GRU1 = layers.Bidirectional(layers.GRU(units=hidden, return_sequences=False))(x)
        out_put = layers.Dense(units=7, activation='softmax')(GRU1)
        model = models.Model(inputs=input_tensor, outputs=[out_put])
        model.summary()
        filepath='stabc.best.hdf5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True,mode='max')
        callbacks_list = [checkpoint]
        model.compile(optimizer='Nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        train_history=model.fit(train_X, train_label, validation_data=(test_X, test_label), epochs=epochs, verbose=2, callbacks=callbacks_list)
        c=np.array(train_history.history['val_categorical_accuracy'])
        d=np.array(train_history.history['val_loss'])
    t=t+1

hi128=(E[0,0,:]+E[0,1,:]+E[0,2,:])/3
hi256=(E[1,0,:]+E[1,1,:]+E[1,2,:])/3
hi512=(E[2,0,:]+E[2,1,:]+E[2,2,:])/3
print(max(hi128),max(hi256),max(hi512))
lhi128=(F[0,0,:]+F[0,1,:]+F[0,2,:])/3
lhi256=(F[1,0,:]+F[1,1,:]+F[1,2,:])/3
lhi512=(F[2,0,:]+F[2,1,:]+F[2,2,:])/3
print(min(lhi128),min(lhi256),min(lhi512))

for i  in range(3):
        model = models.Sequential()
        keras.backend.clear_session() 
        input_tensor = Input(shape=(12,12))
        attention_mul = attention_3d_block(input_tensor)
        x = stabc(nb_filters=filter_nums, kernel_size=kernel_size, dilations=[1, 2], return_sequences=True)(attention_mul)
        GRU1 = layers.Bidirectional(layers.GRU(units=128, return_sequences=True))(x)
        drop1 = Dropout(0.5)(GRU1)
        GRU2 = layers.Bidirectional(layers.GRU(units=64, return_sequences=False))(drop1)
        drop2 = Dropout(0.5)(GRU2)
        out_put = layers.Dense(units=7, activation='softmax')(GRU2)
        model = models.Model(inputs=input_tensor, outputs=[out_put])
        model.summary()
        filepath='stabc.best.hdf5'
        checkpoint = ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True,mode='max')
        callbacks_list = [checkpoint]
        model.compile(optimizer='Nadam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
        train_history=model.fit(train_X, train_label, validation_data=(test_X, test_label), epochs=epochs, verbose=2, callbacks=callbacks_list)
        c=np.array(train_history.history['val_categorical_accuracy'])
        d=np.array(train_history.history['val_loss'])

hi6464=(G[0,0,:]+G[0,1,:]+G[0,2,:])/3
hi12864=(G[1,0,:]+G[1,1,:]+G[1,2,:])/3
hi128128=(G[2,0,:]+G[2,1,:]+G[2,2,:])/3
print(max(hi6464),max(hi12864),max(hi128128))
lhi6464=(H[0,0,:]+H[0,1,:]+H[0,2,:])/3
lhi12864=(H[1,0,:]+H[1,1,:]+H[1,2,:])/3
lhi128128=(H[2,0,:]+H[2,1,:]+H[2,2,:])/3
print(min(lhi6464),min(lhi12864),min(lhi128128))

def lizhu(x):
    print('lizhubieku')
    print('x=x+1')
    return x

print(lizhu(2))